In [25]:
import pandas as pd
from sqlalchemy import create_engine, func, Table, MetaData, inspect, String, Integer
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base
import matplotlib.pyplot as plt

In [26]:
def deliveroo_kapsalon():
    DATABASE_URL = 'sqlite:///databases/deliveroo.db'
    engine = create_engine(DATABASE_URL)

    metadata = MetaData()
    locations_to_restaurants = Table('locations_to_restaurants', metadata, autoload_with=engine)

    Base = automap_base()
    Base.prepare(autoload_with=engine)
    restaurants = Base.classes.restaurants
    menu_item = Base.classes.menu_items
    locations = Base.classes.locations

    Session = sessionmaker(bind=engine)
    session = Session()

 
    query = session.query(
        restaurants.name.label('restaurant_name'),
        func.avg(menu_item.price).label('avg_price'),
        func.min(locations.latitude).label('latitude'),  
        func.min(locations.longitude).label('longitude')  
    ).select_from(menu_item). \
        join(restaurants, restaurants.id == menu_item.restaurant_id). \
        join(locations_to_restaurants, locations_to_restaurants.c.restaurant_id == restaurants.id). \
        join(locations, locations.id == locations_to_restaurants.c.location_id). \
        filter(menu_item.name.like('%kapsalon%')). \
        group_by(
            restaurants.name
        )

    result = query.all()
    results = []
    if result:
        for row in result:
            results.append({
                'restaurant_name': row.restaurant_name,
                'avg_price': row.avg_price, 
                'latitude': row.latitude,
                'longitude': row.longitude
            })
    else:
        print("No results found.")
    
    return results 

In [27]:
def ubereats_kapsalon():
    DATABASE_URL = 'sqlite:///databases/ubereats.db'
    engine = create_engine(DATABASE_URL)
    
    metadata = MetaData()

    try:
        restaurants = Table('restaurants', metadata, autoload_with=engine)
        menu_item = Table('menu_items', metadata, autoload_with=engine)
        locations = Table('locations', metadata, autoload_with=engine)
        locations_to_restaurants = Table('locations_to_restaurants', metadata, autoload_with=engine)
    except Exception as e:
        print(f"Error loading tables: {e}")
        return []  

    Session = sessionmaker(bind=engine)
    session = Session()

    print(f"Restaurants Columns: {[column.name for column in restaurants.columns]}")
    print(f"Menu Items Columns: {[column.name for column in menu_item.columns]}")
    print(f"Locations Columns: {[column.name for column in locations.columns]}")

    try:
        query = session.query(
            restaurants.c.title.label('restaurant_name'),
            (func.avg(menu_item.c.price) / 100).label('avg_price'),  
            func.min(locations.c.latitude).label('latitude'),
            func.min(locations.c.longitude).label('longitude')
        ).select_from(menu_item). \
            join(restaurants, restaurants.c.id == menu_item.c.restaurant_id). \
            join(locations_to_restaurants, locations_to_restaurants.c.restaurant_id == restaurants.c.id). \
            join(locations, locations.c.id == locations_to_restaurants.c.location_id). \
            filter(menu_item.c.name.like('%kapsalon%')). \
            group_by(restaurants.c.title)

        result = query.all()
        results = []
        if result:
            for row in result:
               results.append({
                    'restaurant_name': row.restaurant_name,
                    'avg_price': row.avg_price,
                    'latitude': row.latitude,
                    'longitude': row.longitude
                })
        else:
            print("No results found.")

    except Exception as e:
        print(f"Error executing query: {e}")
        return []

    return results



In [28]:
def takeaway_kapsalon():
    DATABASE_URL = 'sqlite:///databases/takeaway.db'
    engine = create_engine(DATABASE_URL)

   
    metadata = MetaData()

   
    try:
        restaurants = Table('restaurants', metadata, autoload_with=engine)
        menu_item = Table('menuItems', metadata, autoload_with=engine)
        locations = Table('locations', metadata, autoload_with=engine)
        locations_to_restaurants = Table('locations_to_restaurants', metadata, autoload_with=engine)
    except Exception as e:
        print(f"Error loading tables: {e}")
        return

    Session = sessionmaker(bind=engine)
    session = Session()


    print(f"Restaurants Columns: {[column.name for column in restaurants.columns]}")
    print(f"Menu Items Columns: {[column.name for column in menu_item.columns]}")
    print(f"Locations Columns: {[column.name for column in locations.columns]}")
    
    try:
        query = session.query(
            restaurants.c.name.label('restaurant_name'),
            func.avg(menu_item.c.price).label('avg_price'),
            func.min(locations.c.latitude).label('latitude'),
            func.min(locations.c.longitude).label('longitude')
        ).select_from(menu_item). \
            join(restaurants, restaurants.c.primarySlug == menu_item.c.primarySlug). \
            join(locations_to_restaurants, locations_to_restaurants.c.restaurant_id == restaurants.c.primarySlug). \
            join(locations, locations.c.ID == locations_to_restaurants.c.location_id). \
            filter(menu_item.c.name.like('%kapsalon%')). \
            group_by(restaurants.c.name)

        result = query.all()
        results = []
        if result:
            for row in result:
               results.append({
                    'restaurant_name': row.restaurant_name,
                    'avg_price': row.avg_price,
                    'latitude': row.latitude,
                    'longitude': row.longitude
                })
        else:
            print("No results found.")

    except Exception as e:
        print(f"Error executing query: {e}")
        return []

    return results

In [29]:
deliveroo_results = deliveroo_kapsalon()
ubereats_results = ubereats_kapsalon()
takeaway_results = takeaway_kapsalon()
merged_results = deliveroo_results + ubereats_results + takeaway_results

df = pd.DataFrame(merged_results)
df = df.drop_duplicates()
print(df)

Restaurants Columns: ['id', 'visited_time', 'title', 'slug', 'cityslug', 'location__address', 'location__street_address', 'location__city', 'location__country', 'location__postal_code', 'location__region', 'location__latitude', 'location__longitude', 'location__geo__city', 'location__geo__country', 'location__geo__neighborhood', 'location__geo__region', 'location__location_type', 'is_delivery_third_party', 'is_delivery_over_the_top', 'rating__rating_value', 'rating__review_count', 'sanitized_title', 'city_id', 'is_delivery_bandwagon', 'menu_uuid', 'menu_display_type', 'has_multiple_menus', 'parent_chain__uuid', 'parent_chain__name']
Menu Items Columns: ['id', 'restaurant_id', 'menu_section_id', 'name', 'description', 'price', 'display_type', 'is_sold_out', 'has_customizations', 'subsection_uuid', 'is_available', 'price_tagline__accessibility_text']
Locations Columns: ['id', 'name', 'region', 'latitude', 'longitude', 'visited_time']
Restaurants Columns: ['primarySlug', 'restaurant_id', 

In [30]:
class DataBaseManager():
    def __init__(self,db_urls) -> None:
        self.db_data = {}
        self.db_urls = db_urls
        for db_name, db_url in db_urls.items():
            self.db_data[db_name] = {'engine': None, 'session': None, 'tables': {}}
            engine = create_engine(db_url, echo=False)
            inspector = inspect(engine)
            self.db_data[db_name]['engine'] = engine
            Session = sessionmaker(bind=engine)
            self.db_data[db_name]['session'] = Session()
            Base = automap_base()
            Base.prepare(autoload_with=engine)
            tabel_list = inspector.get_table_names()
            match db_name:
                case 'ubereats':
                    metadata = MetaData()
                    for tabel in tabel_list:
                        self.db_data[db_name]['tables'][tabel] =   Table(f'{tabel}', metadata, autoload_with=engine)
                case _:
                    base_tabel_list = Base.classes.keys()
                    for tabel in base_tabel_list:
                        self.db_data[db_name]['tables'][tabel] = Base.classes[f'{tabel}']
                    metadata = MetaData()
                    many_to_many_list = [x for x in tabel_list if x not in base_tabel_list]
                    for tabel in many_to_many_list:
                        self.db_data[db_name]['tables'][tabel] = Table(f'{tabel}',metadata,autoload_with=engine)

    def get_session(self,db_name):
        return self.db_data[db_name]['session']
    
    def get_tables(self,db_name):
        return self.db_data[db_name]['tables']

In [31]:
def query_prices_per_db(self, db_name='ubereats'):
        session = self.get_session(db_name)
        tables = self.get_tables(db_name)
        match db_name:
            case 'ubereats':
                menu_items = tables['menu_items']
                query = session.query(menu_items.c.price)
            case 'takeaway':
                menu_items = tables['menuItems']
                query = session.query(menu_items.price)
            case 'deliveroo':
                menu_items = tables['menu_items']
                query = session.query(menu_items.price)
            case _:
                raise ValueError(f"Unsupported database: {db_name}")
        
        prices = [row.price for row in query.all()]
        session.close()
        return prices

def create_prices_df_for_all_db(self):
        prices_dict = {}
        prices_dict['ubereats'] = self.query_prices_per_db(db_name='ubereats')
        prices_dict['takeaway'] = self.query_prices_per_db(db_name='takeaway')
        prices_dict['deliveroo'] = self.query_prices_per_db(db_name='deliveroo')
        
        prices_df = pd.DataFrame({key: pd.Series(value) for key, value in prices_dict.items()})
        return prices_df

def save_prices_to_csv(self, file_name='prices.csv'):
        df = self.create_prices_df_for_all_db()
        df.to_csv(file_name, index=False)
        print(f"Prices saved to {file_name}")





In [ ]:
df = pd.read_csv('C:/Users/becod/Documents/GitHub/delivery_market_analysis_with_SQL/data_for_plots/prices.csv')
df = df.apply(pd.to_numeric, errors='coerce')
df_clean = df.dropna(how='any')
df_long = df_clean.melt(var_name='Platform', value_name='Price (in Euros)')


price_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]  # Adjust the upper range as needed
price_labels = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100']

df_long['Price Range'] = pd.cut(df_long['Price (in Euros)'], bins=price_bins, labels=price_labels, right=False)

fig = px.histogram(
    df_long, 
    x='Price Range',  # Use the price range as the x-axis
    color='Platform',  # Color by platform (different colors for each platform)
    title="Price Distribution per Platform of Menu Items",
    labels={"Price Range": "Price Range (in Euros)", "count": "Frequency"},  # Adjust labels
    category_orders={"Price Range": price_labels},  # Ensure the order of price ranges is correct
    barmode='group',  # Group bars by platform
    color_discrete_map={
        'ubereats': 'blue',
        'takeaway': 'green',
        'deliveroo': 'red'
    }  # Assign specific colors to platforms
)

# Show the plot
fig.update_layout(
    xaxis_title="Price Range (in Euros)",  # X-axis label
    yaxis_title="Frequency",  # Y-axis label (frequency of items in each range)
    width=800,
    height=600
)

fig.show()